In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config
import warnings
warnings.simplefilter('ignore')

True


In [2]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#train_csv: str = "~/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [3]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
row = df.sample(n=1).iloc[0]
X, Y = generate_batch(df, process_row, batch_size)

In [5]:
Y.shape

(32, 64, 98)

In [6]:
X.shape

(32, 67267)

In [7]:
df.sample(n=1).iloc[0]

dataset                                                    train
start                                            00:01:12.005339
end                                              00:01:16.209543
interval_id                                                  951
pose_fn        Gestures/shelly/train/npz/951-00:01:12.005339-...
audio_fn       Gestures/shelly/train/audio/951-00:00:38.63863...
video_fn       18._The_badness_of_death_Part_III_Immortality_...
speaker                                                   shelly
Name: 1068, dtype: object

In [8]:
from audio_to_multiple_pose_gan.static_model_factory import *
a2p = Audio2Pose(1, 98)
input_dict = {"audio": X}
a2p(input_dict).shape

torch.Size([32, 64, 98])

In [9]:
a2p = Audio2PoseGANS(1, 98)
input_dict = {"audio": X}
a2p(input_dict).shape

torch.Size([32, 64, 98])

In [10]:
Y.shape

(32, 64, 98)

In [11]:
d_patchgan = D_patchgan(in_channels=64)

In [12]:
Y.shape

(32, 64, 98)

In [14]:
Y.shape

(32, 64, 98)

In [15]:
from audio_to_multiple_pose_gan.torch_layers import *
to_motion_delta(Y).shape

(32, 63, 98)

In [16]:
Y = torch.tensor(Y).float()

In [17]:
a2p = Audio2PoseGANS(1, 98)
input_dict = {"audio": X}
pred_Y = a2p(input_dict)

In [18]:
keypoints_regloss(Y, pred_Y, regloss_type="l1")

tensor(0.8104, grad_fn=<MeanBackward0>)

In [27]:
from audio_to_multiple_pose_gan.config import get_config
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
#get_modelの実装は任せる
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANS, D_patchgan
from audio_to_multiple_pose_gan.torch_layers import to_motion_delta, keypoints_to_train, keypoints_regloss
from common.audio_lib import save_audio_sample
from common.audio_repr import raw_repr
from common.consts import RIGHT_BODY_KEYPOINTS, LEFT_BODY_KEYPOINTS, LEFT_HAND_KEYPOINTS, \
    RIGHT_HAND_KEYPOINTS, POSE_SAMPLE_SHAPE, G_SCOPE, D_SCOPE, SR, FRAMES_PER_SAMPLE, AUDIO_SHAPE
from common.evaluation import compute_pck
from common.pose_logic_lib import translate_keypoints, get_sample_output_by_config
from common.pose_plot_lib import save_side_by_side_video, save_video_from_audio_video
from torch import optim
import torch
import pandas as pd
import argparse
from torch import nn

def get_training_keypoints():
    training_keypoints = []
    training_keypoints.extend(RIGHT_BODY_KEYPOINTS)
    training_keypoints.extend(LEFT_BODY_KEYPOINTS)
    for i in range(5):
        training_keypoints.extend(RIGHT_HAND_KEYPOINTS(i))
        training_keypoints.extend(LEFT_HAND_KEYPOINTS(i))
    training_keypoints = sorted(list(set(training_keypoints)))
    return training_keypoints

In [30]:
training_keypoints = get_training_keypoints()
keypoints_to_train(Y, torch.LongTensor(training_keypoints))

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [32]:
torch.LongTensor(training_keypoints).shape

torch.Size([48])

In [33]:
Y.shape

torch.Size([32, 64, 98])

In [25]:
d_patchgan = D_patchgan(in_channels=127)
d_patchgan(torch.concatenate([Y,to_motion_delta(Y)],dim=1))

tensor([[0.4711],
        [0.4701],
        [0.4643],
        [0.5149],
        [0.5422],
        [0.4667],
        [0.5427],
        [0.4738],
        [0.3526],
        [0.4910],
        [0.4615],
        [0.4670],
        [0.5105],
        [0.5141],
        [0.4553],
        [0.4931],
        [0.4335],
        [0.5136],
        [0.4432],
        [0.5083],
        [0.5133],
        [0.4874],
        [0.4726],
        [0.5026],
        [0.5000],
        [0.4595],
        [0.4312],
        [0.5144],
        [0.4437],
        [0.5043],
        [0.4706],
        [0.4842]], grad_fn=<SigmoidBackward0>)

In [34]:
torch.hann_window(window_length=30)

tensor([0.0000, 0.0109, 0.0432, 0.0955, 0.1654, 0.2500, 0.3455, 0.4477, 0.5523,
        0.6545, 0.7500, 0.8346, 0.9045, 0.9568, 0.9891, 1.0000, 0.9891, 0.9568,
        0.9045, 0.8346, 0.7500, 0.6545, 0.5523, 0.4477, 0.3455, 0.2500, 0.1654,
        0.0955, 0.0432, 0.0109])